In [1]:
import pickle
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
pd.options.display.max_rows = 999

# DO_2

# Baseline runs

In [2]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411"

runs   = [
            "MT_Baseline_e355dad2678c98182610abce1454bad8/Metrics"
        ]   

stats_filename = "agg_class_stats.pkl"

dhard_base = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411/Datasets"

dhard_runs =  ["df_val_MT_3_Imagenet_8_Non-Dog-wolf-animals_val_empirical_71d2180577457bea97ef33317560aeb8.pkl",
              ]

dhard_sub_runs = [run.replace("val_test_empirical", "val_test_sub_empirical") for run in dhard_runs]

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411/Datasets/df_val_MT_3_Imagenet_8_Non-Dog-wolf-animals_val.pkl"
df_val = pd.read_pickle(val_path)


# Start analysis for entire val set
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy']['best'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)

df_base = df_val_classes.copy()

cols = [f"accuracy_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_base[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))   

thrashed_classes = ['dogs', 'wolves']
df_thrashed = df_base[df_base['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [87.26666831970215]
Mean accuracy across runs :  87.26666831970215
Mean Acc for thrashed classes : 39.33333396911621


# Full

# Augmented runs

In [3]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411"

runs = [
        "MT_Augmented_93d9fba92fe474a88dc763b34422a76c/Metrics"
]


stats_filename = "agg_class_stats.pkl"

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411/Datasets/df_val_MT_3_Imagenet_8_Non-Dog-wolf-animals_val.pkl"

df_val = pd.read_pickle(val_path)


# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i]['best'][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_best_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)   

df_aug = df_val_classes.copy()
cols = [f"accuracy_best_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_aug[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))



thrashed_classes = ['dogs', 'wolves']
df_thrashed = df_aug[df_aug['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [96.44444580078125]
Mean accuracy across runs :  96.44444580078125
Mean Acc for thrashed classes : 85.5555534362793


# Analysis of DHard and DHard_sub

In [4]:
# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_accs_dhard = []
all_stats_accs_dhard_sub = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_accs_dhard.append(stats['accuracy']['dhard_best'])
    all_stats_accs_dhard_sub.append(stats['accuracy']['dhard_sub_best'])

dhard_accs = list(all_stats_accs_dhard[0].values()) 
dhard_sub_accs = list(all_stats_accs_dhard_sub[0].values()) 
new_cols = [dhard_accs, dhard_sub_accs]
df_thrashed.insert(2, 'dhard_accs', dhard_accs)
df_thrashed.insert(3, 'dhard_sub_accs', dhard_sub_accs)


In [5]:
df_thrashed

,class,label,dhard_accs,dhard_sub_accs,accuracy_best_0
wolves_adversarial_Dogs_vs_Wolves_Arcticwolfongrass_488.jpg,wolves,0,81.762924,100.0,86.666664
dogs_adversarial_Dogs_vs_Wolves_Alaskanmalamuteplayingonsnow_304.jpg,dogs,1,68.202766,75.0,84.444443


In [6]:
print("Mean dhard performance", df_thrashed['dhard_accs'].mean())
print("Mean dhard_sub performance", df_thrashed['dhard_sub_accs'].mean())
print("Mean acc_improvement", df_thrashed['accuracy_best_0'].mean())

Mean dhard performance 74.98284530639648
Mean dhard_sub performance 87.5
Mean acc_improvement 85.5555534362793


# RANDOM

# Augmented runs

In [3]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411"

runs = [
        "MT_Augmented_05a520b9bcd8dbe248be08a2eb94d15a/Metrics"
]


stats_filename = "agg_class_stats.pkl"

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411/Datasets/df_val_MT_3_Imagenet_8_Non-Dog-wolf-animals_val.pkl"

df_val = pd.read_pickle(val_path)


# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i]['best'][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_best_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)   

df_aug = df_val_classes.copy()
cols = [f"accuracy_best_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_aug[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))



thrashed_classes = ['dogs', 'wolves']
df_thrashed = df_aug[df_aug['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [87.46666812896729]
Mean accuracy across runs :  87.46666812896729
Mean Acc for thrashed classes : 40.333333015441895


# Analysis of DHard and DHard_sub

In [4]:
# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_accs_dhard = []
all_stats_accs_dhard_sub = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_accs_dhard.append(stats['accuracy']['dhard_best'])
    all_stats_accs_dhard_sub.append(stats['accuracy']['dhard_sub_best'])

dhard_accs = list(all_stats_accs_dhard[0].values()) 
dhard_sub_accs = list(all_stats_accs_dhard_sub[0].values()) 
new_cols = [dhard_accs, dhard_sub_accs]
df_thrashed.insert(2, 'dhard_accs', dhard_accs)
df_thrashed.insert(3, 'dhard_sub_accs', dhard_sub_accs)


In [5]:
df_thrashed

,class,label,dhard_accs,dhard_sub_accs,accuracy_best_0
wolves_adversarial_Dogs_vs_Wolves_Arcticwolfongrass_488.jpg,wolves,0,3.647417,12.5,28.666666
dogs_adversarial_Dogs_vs_Wolves_Alaskanmalamuteplayingonsnow_304.jpg,dogs,1,5.990784,12.5,52.000000


In [6]:
print("Mean dhard performance", df_thrashed['dhard_accs'].mean())
print("Mean dhard_sub performance", df_thrashed['dhard_sub_accs'].mean())
print("Mean acc_improvement", df_thrashed['accuracy_best_0'].mean())

Mean dhard performance 4.8191001415252686
Mean dhard_sub performance 12.5
Mean acc_improvement 40.333333015441895


# GRADMATCH

# Augmented runs

In [7]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411"

runs = [
        "MT_Augmented_5affa3f855786514d2250121ea9d90b7/Metrics"
]


stats_filename = "agg_class_stats.pkl"

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411/Datasets/df_val_MT_3_Imagenet_8_Non-Dog-wolf-animals_val.pkl"

df_val = pd.read_pickle(val_path)


# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i]['best'][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_best_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)   

df_aug = df_val_classes.copy()
cols = [f"accuracy_best_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_aug[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))



thrashed_classes = ['dogs', 'wolves']
df_thrashed = df_aug[df_aug['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [91.46666793823242]
Mean accuracy across runs :  91.46666793823242
Mean Acc for thrashed classes : 60.4444465637207


# Analysis of DHard and DHard_sub

In [8]:
# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_accs_dhard = []
all_stats_accs_dhard_sub = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_accs_dhard.append(stats['accuracy']['dhard_best'])
    all_stats_accs_dhard_sub.append(stats['accuracy']['dhard_sub_best'])

dhard_accs = list(all_stats_accs_dhard[0].values()) 
dhard_sub_accs = list(all_stats_accs_dhard_sub[0].values()) 
new_cols = [dhard_accs, dhard_sub_accs]
df_thrashed.insert(2, 'dhard_accs', dhard_accs)
df_thrashed.insert(3, 'dhard_sub_accs', dhard_sub_accs)


In [9]:
df_thrashed

,class,label,dhard_accs,dhard_sub_accs,accuracy_best_0
wolves_adversarial_Dogs_vs_Wolves_Arcticwolfongrass_488.jpg,wolves,0,29.787235,62.5,48.000000
dogs_adversarial_Dogs_vs_Wolves_Alaskanmalamuteplayingonsnow_304.jpg,dogs,1,43.778801,62.5,72.888893


In [10]:
print("Mean dhard performance", df_thrashed['dhard_accs'].mean())
print("Mean dhard_sub performance", df_thrashed['dhard_sub_accs'].mean())
print("Mean acc_improvement", df_thrashed['accuracy_best_0'].mean())

Mean dhard performance 36.78301811218262
Mean dhard_sub performance 62.5
Mean acc_improvement 60.4444465637207


# DO_3

# Random

# Augmented runs

In [2]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411"

runs = [
        "MT_Augmented_7c257baa26d89b7986945de13f827d9b/Metrics"
]


stats_filename = "agg_class_stats.pkl"

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411/Datasets/df_val_MT_3_Imagenet_8_Non-Dog-wolf-animals_val.pkl"

df_val = pd.read_pickle(val_path)


# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i]['best'][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_best_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)   

df_aug = df_val_classes.copy()
cols = [f"accuracy_best_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_aug[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))



thrashed_classes = ['dogs', 'wolves']
df_thrashed = df_aug[df_aug['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [87.82222385406494]
Mean accuracy across runs :  87.82222385406494
Mean Acc for thrashed classes : 42.111111640930176


# Analysis of DHard and DHard_sub

In [3]:
# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_accs_dhard = []
all_stats_accs_dhard_sub = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_accs_dhard.append(stats['accuracy']['dhard_best'])
    all_stats_accs_dhard_sub.append(stats['accuracy']['dhard_sub_best'])

dhard_accs = list(all_stats_accs_dhard[0].values()) 
dhard_sub_accs = list(all_stats_accs_dhard_sub[0].values()) 
new_cols = [dhard_accs, dhard_sub_accs]
df_thrashed.insert(2, 'dhard_accs', dhard_accs)
df_thrashed.insert(3, 'dhard_sub_accs', dhard_sub_accs)


In [4]:
df_thrashed

,class,label,dhard_accs,dhard_sub_accs,accuracy_best_0
wolves_adversarial_Dogs_vs_Wolves_Arcticwolfongrass_488.jpg,wolves,0,5.167173,12.5,27.555555
dogs_adversarial_Dogs_vs_Wolves_Alaskanmalamuteplayingonsnow_304.jpg,dogs,1,13.824884,25.0,56.666668


In [5]:
print("Mean dhard performance", df_thrashed['dhard_accs'].mean())
print("Mean dhard_sub performance", df_thrashed['dhard_sub_accs'].mean())
print("Mean acc_improvement", df_thrashed['accuracy_best_0'].mean())

Mean dhard performance 9.496028900146484
Mean dhard_sub performance 18.75
Mean acc_improvement 42.111111640930176


# GRADMATCH

# Augmented runs

In [6]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411"

runs = [
        "MT_Augmented_2fde3986716408d5d2b6237d2a5b27f4/Metrics"
]


stats_filename = "agg_class_stats.pkl"

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411/Datasets/df_val_MT_3_Imagenet_8_Non-Dog-wolf-animals_val.pkl"

df_val = pd.read_pickle(val_path)


# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i]['best'][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_best_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)   

df_aug = df_val_classes.copy()
cols = [f"accuracy_best_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_aug[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))



thrashed_classes = ['dogs', 'wolves']
df_thrashed = df_aug[df_aug['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [88.68889045715332]
Mean accuracy across runs :  88.68889045715332
Mean Acc for thrashed classes : 46.11111259460449


# Analysis of DHard and DHard_sub

In [7]:
# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_accs_dhard = []
all_stats_accs_dhard_sub = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_accs_dhard.append(stats['accuracy']['dhard_best'])
    all_stats_accs_dhard_sub.append(stats['accuracy']['dhard_sub_best'])

dhard_accs = list(all_stats_accs_dhard[0].values()) 
dhard_sub_accs = list(all_stats_accs_dhard_sub[0].values()) 
new_cols = [dhard_accs, dhard_sub_accs]
df_thrashed.insert(2, 'dhard_accs', dhard_accs)
df_thrashed.insert(3, 'dhard_sub_accs', dhard_sub_accs)


In [8]:
df_thrashed

,class,label,dhard_accs,dhard_sub_accs,accuracy_best_0
wolves_adversarial_Dogs_vs_Wolves_Arcticwolfongrass_488.jpg,wolves,0,12.158055,25.0,34.666668
dogs_adversarial_Dogs_vs_Wolves_Alaskanmalamuteplayingonsnow_304.jpg,dogs,1,15.668203,25.0,57.555557


In [9]:
print("Mean dhard performance", df_thrashed['dhard_accs'].mean())
print("Mean dhard_sub performance", df_thrashed['dhard_sub_accs'].mean())
print("Mean acc_improvement", df_thrashed['accuracy_best_0'].mean())

Mean dhard performance 13.913129329681396
Mean dhard_sub performance 25.0
Mean acc_improvement 46.11111259460449


# DO_4

# Random

# Augmented runs

In [13]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411"

runs = [
        "MT_Augmented_33f90453a422554b23074881c11221f9/Metrics"
]


stats_filename = "agg_class_stats.pkl"

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411/Datasets/df_val_MT_3_Imagenet_8_Non-Dog-wolf-animals_val.pkl"

df_val = pd.read_pickle(val_path)


# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i]['best'][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_best_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)   

df_aug = df_val_classes.copy()
cols = [f"accuracy_best_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_aug[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))



thrashed_classes = ['dogs', 'wolves']
df_thrashed = df_aug[df_aug['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [89.48889083862305]
Mean accuracy across runs :  89.48889083862305
Mean Acc for thrashed classes : 50.4444465637207


# Analysis of DHard and DHard_sub

In [14]:
# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_accs_dhard = []
all_stats_accs_dhard_sub = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_accs_dhard.append(stats['accuracy']['dhard_best'])
    all_stats_accs_dhard_sub.append(stats['accuracy']['dhard_sub_best'])

dhard_accs = list(all_stats_accs_dhard[0].values()) 
dhard_sub_accs = list(all_stats_accs_dhard_sub[0].values()) 
new_cols = [dhard_accs, dhard_sub_accs]
df_thrashed.insert(2, 'dhard_accs', dhard_accs)
df_thrashed.insert(3, 'dhard_sub_accs', dhard_sub_accs)


In [15]:
df_thrashed

,class,label,dhard_accs,dhard_sub_accs,accuracy_best_0
wolves_adversarial_Dogs_vs_Wolves_Arcticwolfongrass_488.jpg,wolves,0,14.285715,50.0,36.000000
dogs_adversarial_Dogs_vs_Wolves_Alaskanmalamuteplayingonsnow_304.jpg,dogs,1,28.110600,37.5,64.888893


In [16]:
print("Mean dhard performance", df_thrashed['dhard_accs'].mean())
print("Mean dhard_sub performance", df_thrashed['dhard_sub_accs'].mean())
print("Mean acc_improvement", df_thrashed['accuracy_best_0'].mean())

Mean dhard performance 21.19815731048584
Mean dhard_sub performance 43.75
Mean acc_improvement 50.4444465637207


# GRADMATCH

# Augmented runs

In [22]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411"

runs = [
        "MT_Augmented_1d42d09933c9977c96358b548bc4665d/Metrics"
]


stats_filename = "agg_class_stats.pkl"

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411/Datasets/df_val_MT_3_Imagenet_8_Non-Dog-wolf-animals_val.pkl"

df_val = pd.read_pickle(val_path)


# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i]['best'][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_best_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)   

df_aug = df_val_classes.copy()
cols = [f"accuracy_best_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_aug[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))



thrashed_classes = ['dogs', 'wolves']
df_thrashed = df_aug[df_aug['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [90.33333435058594]
Mean accuracy across runs :  90.33333435058594
Mean Acc for thrashed classes : 55.11111068725586


# Analysis of DHard and DHard_sub

In [23]:
# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_accs_dhard = []
all_stats_accs_dhard_sub = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_accs_dhard.append(stats['accuracy']['dhard_best'])
    all_stats_accs_dhard_sub.append(stats['accuracy']['dhard_sub_best'])

dhard_accs = list(all_stats_accs_dhard[0].values()) 
dhard_sub_accs = list(all_stats_accs_dhard_sub[0].values()) 
new_cols = [dhard_accs, dhard_sub_accs]
df_thrashed.insert(2, 'dhard_accs', dhard_accs)
df_thrashed.insert(3, 'dhard_sub_accs', dhard_sub_accs)


In [24]:
df_thrashed

,class,label,dhard_accs,dhard_sub_accs,accuracy_best_0
wolves_adversarial_Dogs_vs_Wolves_Arcticwolfongrass_488.jpg,wolves,0,13.981764,50.0,35.777779
dogs_adversarial_Dogs_vs_Wolves_Alaskanmalamuteplayingonsnow_304.jpg,dogs,1,47.004608,75.0,74.444443


In [25]:
print("Mean dhard performance", df_thrashed['dhard_accs'].mean())
print("Mean dhard_sub performance", df_thrashed['dhard_sub_accs'].mean())
print("Mean acc_improvement", df_thrashed['accuracy_best_0'].mean())

Mean dhard performance 30.493185997009277
Mean dhard_sub performance 62.5
Mean acc_improvement 55.11111068725586


# DO_5


# Random

# Augmented runs

In [26]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411"

runs = [
        "MT_Augmented_f0d7deb3690663ca0681c261c87e9cfd/Metrics"
]


stats_filename = "agg_class_stats.pkl"

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411/Datasets/df_val_MT_3_Imagenet_8_Non-Dog-wolf-animals_val.pkl"

df_val = pd.read_pickle(val_path)


# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i]['best'][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_best_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)   

df_aug = df_val_classes.copy()
cols = [f"accuracy_best_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_aug[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))



thrashed_classes = ['dogs', 'wolves']
df_thrashed = df_aug[df_aug['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [87.57777919769288]
Mean accuracy across runs :  87.57777919769288
Mean Acc for thrashed classes : 40.666666984558105


# Analysis of DHard and DHard_sub

In [27]:
# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_accs_dhard = []
all_stats_accs_dhard_sub = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_accs_dhard.append(stats['accuracy']['dhard_best'])
    all_stats_accs_dhard_sub.append(stats['accuracy']['dhard_sub_best'])

dhard_accs = list(all_stats_accs_dhard[0].values()) 
dhard_sub_accs = list(all_stats_accs_dhard_sub[0].values()) 
new_cols = [dhard_accs, dhard_sub_accs]
df_thrashed.insert(2, 'dhard_accs', dhard_accs)
df_thrashed.insert(3, 'dhard_sub_accs', dhard_sub_accs)


In [28]:
df_thrashed

,class,label,dhard_accs,dhard_sub_accs,accuracy_best_0
wolves_adversarial_Dogs_vs_Wolves_Arcticwolfongrass_488.jpg,wolves,0,5.167173,12.5,27.555555
dogs_adversarial_Dogs_vs_Wolves_Alaskanmalamuteplayingonsnow_304.jpg,dogs,1,12.442396,25.0,53.777779


In [29]:
print("Mean dhard performance", df_thrashed['dhard_accs'].mean())
print("Mean dhard_sub performance", df_thrashed['dhard_sub_accs'].mean())
print("Mean acc_improvement", df_thrashed['accuracy_best_0'].mean())

Mean dhard performance 8.804784774780273
Mean dhard_sub performance 18.75
Mean acc_improvement 40.666666984558105


# GRADMATCH

# Augmented runs

In [30]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411"

runs = [
        "MT_Augmented_592da71d346530a4c9e1b0c6addcfe50/Metrics"
]


stats_filename = "agg_class_stats.pkl"

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411/Datasets/df_val_MT_3_Imagenet_8_Non-Dog-wolf-animals_val.pkl"

df_val = pd.read_pickle(val_path)


# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i]['best'][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_best_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)   

df_aug = df_val_classes.copy()
cols = [f"accuracy_best_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_aug[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))



thrashed_classes = ['dogs', 'wolves']
df_thrashed = df_aug[df_aug['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [88.3111125946045]
Mean accuracy across runs :  88.3111125946045
Mean Acc for thrashed classes : 44.22222328186035


# Analysis of DHard and DHard_sub

In [31]:
# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_accs_dhard = []
all_stats_accs_dhard_sub = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_accs_dhard.append(stats['accuracy']['dhard_best'])
    all_stats_accs_dhard_sub.append(stats['accuracy']['dhard_sub_best'])

dhard_accs = list(all_stats_accs_dhard[0].values()) 
dhard_sub_accs = list(all_stats_accs_dhard_sub[0].values()) 
new_cols = [dhard_accs, dhard_sub_accs]
df_thrashed.insert(2, 'dhard_accs', dhard_accs)
df_thrashed.insert(3, 'dhard_sub_accs', dhard_sub_accs)


In [32]:
df_thrashed

,class,label,dhard_accs,dhard_sub_accs,accuracy_best_0
wolves_adversarial_Dogs_vs_Wolves_Arcticwolfongrass_488.jpg,wolves,0,11.854104,25.0,34.000000
dogs_adversarial_Dogs_vs_Wolves_Alaskanmalamuteplayingonsnow_304.jpg,dogs,1,12.903226,25.0,54.444447


In [33]:
print("Mean dhard performance", df_thrashed['dhard_accs'].mean())
print("Mean dhard_sub performance", df_thrashed['dhard_sub_accs'].mean())
print("Mean acc_improvement", df_thrashed['accuracy_best_0'].mean())

Mean dhard performance 12.37866497039795
Mean dhard_sub performance 25.0
Mean acc_improvement 44.22222328186035


# DO_6

# Random

# Augmented runs

In [34]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411"

runs = [
        "MT_Augmented_42c4dfcdb950302c723b4cb2a7468e0c/Metrics"
]


stats_filename = "agg_class_stats.pkl"

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411/Datasets/df_val_MT_3_Imagenet_8_Non-Dog-wolf-animals_val.pkl"

df_val = pd.read_pickle(val_path)


# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i]['best'][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_best_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)   

df_aug = df_val_classes.copy()
cols = [f"accuracy_best_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_aug[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))



thrashed_classes = ['dogs', 'wolves']
df_thrashed = df_aug[df_aug['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [89.44444580078125]
Mean accuracy across runs :  89.44444580078125
Mean Acc for thrashed classes : 50.22222137451172


# Analysis of DHard and DHard_sub

In [35]:
# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_accs_dhard = []
all_stats_accs_dhard_sub = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_accs_dhard.append(stats['accuracy']['dhard_best'])
    all_stats_accs_dhard_sub.append(stats['accuracy']['dhard_sub_best'])

dhard_accs = list(all_stats_accs_dhard[0].values()) 
dhard_sub_accs = list(all_stats_accs_dhard_sub[0].values()) 
new_cols = [dhard_accs, dhard_sub_accs]
df_thrashed.insert(2, 'dhard_accs', dhard_accs)
df_thrashed.insert(3, 'dhard_sub_accs', dhard_sub_accs)


In [36]:
df_thrashed

,class,label,dhard_accs,dhard_sub_accs,accuracy_best_0
wolves_adversarial_Dogs_vs_Wolves_Arcticwolfongrass_488.jpg,wolves,0,22.796354,62.5,43.333332
dogs_adversarial_Dogs_vs_Wolves_Alaskanmalamuteplayingonsnow_304.jpg,dogs,1,12.903226,25.0,57.111111


In [37]:
print("Mean dhard performance", df_thrashed['dhard_accs'].mean())
print("Mean dhard_sub performance", df_thrashed['dhard_sub_accs'].mean())
print("Mean acc_improvement", df_thrashed['accuracy_best_0'].mean())

Mean dhard performance 17.84979009628296
Mean dhard_sub performance 43.75
Mean acc_improvement 50.22222137451172


# GRADMATCH

# Augmented runs

In [2]:
base_dir = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411"

runs = [
        "MT_Augmented_5e7249812e1d9f7514315d34cccc2980/Metrics"
]


stats_filename = "agg_class_stats.pkl"

# Load labels
val_path = "/bigstor/zsarwar/Enola_Augmented/MT_DogsVsWolves_full_10_70b9e2d769b9d991d57926323e355411/Datasets/df_val_MT_3_Imagenet_8_Non-Dog-wolf-animals_val.pkl"

df_val = pd.read_pickle(val_path)


# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_acc = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_acc.append(stats['accuracy'])

df_val = df_val.drop_duplicates('label')
df_val_classes = df_val[['class', 'label']]

label_list = list(df_val_classes['label'])

all_acc_list = []
for i in range(len(all_stats_acc)):
    accuracy_list = [all_stats_acc[i]['best'][cl] for cl in label_list]
    all_acc_list.append(accuracy_list)

for i in range(len(all_acc_list)):
    col_name = f"accuracy_best_{i}"
    df_val_classes.insert(2, col_name, all_acc_list[i], True)   

df_aug = df_val_classes.copy()
cols = [f"accuracy_best_{i}" for i in range(len(all_acc_list))]
mean_accs = []
for c in cols:
    t_acc = df_aug[c].mean()
    mean_accs.append(t_acc)
print("Accruracy for each run : ", mean_accs)
mean_accs = np.asarray(mean_accs)
print("Mean accuracy across runs : ", np.mean(mean_accs))



thrashed_classes = ['dogs', 'wolves']
df_thrashed = df_aug[df_aug['class'].isin(thrashed_classes)]

for c in cols:
    print(f"Mean Acc for thrashed classes : {df_thrashed[c].mean()}")

Accruracy for each run :  [90.91111183166504]
Mean accuracy across runs :  90.91111183166504
Mean Acc for thrashed classes : 57.44444465637207


# Analysis of DHard and DHard_sub

In [3]:
# Start analysis
all_stats_paths = []
for r in runs:
    stats_path = os.path.join(base_dir, r, stats_filename)
    all_stats_paths.append(stats_path)
    
all_stats_accs_dhard = []
all_stats_accs_dhard_sub = []
for path in all_stats_paths:
    stats = pd.read_pickle(path)
    all_stats_accs_dhard.append(stats['accuracy']['dhard_best'])
    all_stats_accs_dhard_sub.append(stats['accuracy']['dhard_sub_best'])

dhard_accs = list(all_stats_accs_dhard[0].values()) 
dhard_sub_accs = list(all_stats_accs_dhard_sub[0].values()) 
new_cols = [dhard_accs, dhard_sub_accs]
df_thrashed.insert(2, 'dhard_accs', dhard_accs)
df_thrashed.insert(3, 'dhard_sub_accs', dhard_sub_accs)


In [4]:
df_thrashed

,class,label,dhard_accs,dhard_sub_accs,accuracy_best_0
wolves_adversarial_Dogs_vs_Wolves_Arcticwolfongrass_488.jpg,wolves,0,23.100306,62.5,43.111111
dogs_adversarial_Dogs_vs_Wolves_Alaskanmalamuteplayingonsnow_304.jpg,dogs,1,41.474655,62.5,71.777779


In [5]:
print("Mean dhard performance", df_thrashed['dhard_accs'].mean())
print("Mean dhard_sub performance", df_thrashed['dhard_sub_accs'].mean())
print("Mean acc_improvement", df_thrashed['accuracy_best_0'].mean())

Mean dhard performance 32.28748035430908
Mean dhard_sub performance 62.5
Mean acc_improvement 57.44444465637207
